In [1]:
import rasterio as rio
from rasterio import features
import geopandas as gpd
import time
from matplotlib import pyplot as plt
import numpy as np
from helpers import spatial_agg_point_df
from shapely.geometry import Point
from fiona.crs import from_epsg
from glob import glob
import os
import pickle

PyTables is not installed. No support for HDF output.


In [ ]:
shp_fi = r"D:\projects\RD\night_fire\modis_fire_points\DL_FIRE_M6_50643\fire_archive_M6_50643.shp"
df = gpd.read_file(shp_fi)

df_sub = df.query('CONFIDENCE > 50')
daynight = list(df_sub.groupby('DAYNIGHT'))
df_day = daynight[0][1]
df_night = daynight[1][1]


df_night_agg = spatial_agg_point_df(df_night, agg_val=0.5, calc='mean', epsg=4326)
df_night_agg_std = spatial_agg_point_df(df_night, agg_val=0.5, calc='std', epsg=4326)

df_night_agg_count = spatial_agg_point_df(df_night, agg_val=0.5, calc='count', epsg=4326)
df_day_agg_count = spatial_agg_point_df(df_day, agg_val=0.5, calc='count', epsg=4326)

df_night_agg_count['night_perc'] = df_night_agg_count['FRP'] / (df_night_agg_count['FRP'] + df_day_agg_count['FRP'])
df_night_agg_count['night_perc'] *= 100

df_night_agg_count.dropna().plot(column='night_perc', cmap='magma', figsize=(30,20))

In [ ]:
with open('climate_grid_meta_0_25_wgs84.pickle', 'rb') as fp:
    meta = pickle.load(fp)
    meta.update(driver='GTiff')
    


In [ ]:
# write the output file
out_fn = r'D:\projects\RD\misc\test_tile\day_counts_agg_perc.tif'
df_2_write = df_night_agg_count.dropna()
out_arr = np.zeros((meta['height'], meta['width'])).astype('float32')
with rio.open(out_fn, 'w', **meta) as out:

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(df_2_write.geometry, df_2_write.night_perc))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

## now do for the lot of 'em'

In [ ]:
with open('climate_grid_meta_0_25_wgs84.pickle', 'rb') as fp:
    meta = pickle.load(fp)

fire_pts_folders = !ls -d ../modis_fire_points/*
shp_files = [glob(folder + '/*.shp')[0] for folder in fire_pts_folders]

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_no = list(range(1,13))
conf = 0. # confidence threshold
out_folder = r'D:\projects\RD\night_fire\grid_0_25_degree'

# could probably farm out the shapefile
for shp in shp_files:
    
    # read the file
    df = gpd.read_file(shp)
    
    # get month and year info
    df['month'] = [int(acq.split('-')[1]) for acq in df.ACQ_DATE]
    year = int(df.ACQ_DATE[0].split('-')[0])
    
    print('on year {}'.format(year))
    
    # iterate through the months
    for m in df['month'].unique():
        
        # get the month name for filenaming
        month_name = months[m-1]
        
        # subset the dataframe by month and group by daynight
        df_sub = df.query('month == {}'.format(m))
        df_sub = df_sub.query('CONFIDENCE > {}'.format(conf))
        daynight = list(df.groupby('DAYNIGHT'))
        df_day = daynight[0][1]
        df_night = daynight[1][1]
        
        # do the aggregation by count
        df_night_agg_count = spatial_agg_point_df(df_night, agg_val=0.5, calc='count', epsg=4326)
        df_day_agg_count = spatial_agg_point_df(df_day, agg_val=0.5, calc='count', epsg=4326)
        
        # calculate the percentages per cell
        df_night_agg_count['count_perc'] = df_night_agg_count['FRP'] / (df_night_agg_count['FRP'] + df_day_agg_count['FRP'])
        df_night_agg_count['count_perc'] *= 100
        
        df_day_agg_count['count_perc'] = df_day_agg_count['FRP'] / (df_night_agg_count['FRP'] + df_day_agg_count['FRP'])
        df_day_agg_count['count_perc'] *= 100
        
        # write out to raster file
        day_fname = 'modis_{}_countPerc_fire_{}_{}.tif'.format('D', month_name, year)
        night_fname = 'modis_{}_countPerc_fire_{}_{}.tif'.format('N', month_name, year)
        
        # dummy array for holding data (0 will be nodata)
        out_arr = np.zeros((meta['height'], meta['width'])).astype('float32')
        for fname,df_2_write in zip((day_fname, night_fname), (df_day_agg_count.dropna(), df_night_agg_count.dropna())):
        
            out_fn = os.path.join(out_folder, fname)
            with rio.open(out_fn, 'w', **meta) as out:

                # this is where we create a generator of geom, value pairs to use in rasterizing
                shapes = ((geom,value) for geom, value in zip(df_2_write.geometry, df_2_write.count_perc))

                burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
                out.write_band(1, burned)

# use a function to do it

In [2]:
from helpers import create_global_agg_var_grid


In [3]:
with open('climate_grid_meta_0_25_wgs84.pickle', 'rb') as fp:
    meta = pickle.load(fp)

fire_pts_folders = !ls -d ../modis_fire_points/*
shp_files = [glob(folder + '/*.shp')[0] for folder in fire_pts_folders]

create_global_agg_var_grid(shp_files, meta, out_folder=r'D:\projects\RD\night_fire\grid_0_25_degree_vars_2')

on year 2000
writing out November AFC_num rasters
writing out November AFC_perc rasters
writing out November FRP_mean rasters
writing out November FRP_max rasters
writing out November FRP_total rasters
writing out December AFC_num rasters
writing out December AFC_perc rasters
writing out December FRP_mean rasters
writing out December FRP_max rasters
writing out December FRP_total rasters
on year 2001
writing out January AFC_num rasters
writing out January AFC_perc rasters
writing out January FRP_mean rasters
writing out January FRP_max rasters
writing out January FRP_total rasters
writing out February AFC_num rasters
writing out February AFC_perc rasters
writing out February FRP_mean rasters
writing out February FRP_max rasters
writing out February FRP_total rasters
writing out March AFC_num rasters
writing out March AFC_perc rasters
writing out March FRP_mean rasters
writing out March FRP_max rasters
writing out March FRP_total rasters
writing out April AFC_num rasters
writing out Apr

writing out August FRP_mean rasters
writing out August FRP_max rasters
writing out August FRP_total rasters
writing out September AFC_num rasters
writing out September AFC_perc rasters
writing out September FRP_mean rasters
writing out September FRP_max rasters
writing out September FRP_total rasters
writing out October AFC_num rasters
writing out October AFC_perc rasters
writing out October FRP_mean rasters
writing out October FRP_max rasters
writing out October FRP_total rasters
writing out November AFC_num rasters
writing out November AFC_perc rasters
writing out November FRP_mean rasters
writing out November FRP_max rasters
writing out November FRP_total rasters
writing out December AFC_num rasters
writing out December AFC_perc rasters
writing out December FRP_mean rasters
writing out December FRP_max rasters
writing out December FRP_total rasters
on year 2005
writing out January AFC_num rasters
writing out January AFC_perc rasters
writing out January FRP_mean rasters
writing out J

writing out May FRP_mean rasters
writing out May FRP_max rasters
writing out May FRP_total rasters
writing out June AFC_num rasters
writing out June AFC_perc rasters
writing out June FRP_mean rasters
writing out June FRP_max rasters
writing out June FRP_total rasters
writing out July AFC_num rasters
writing out July AFC_perc rasters
writing out July FRP_mean rasters
writing out July FRP_max rasters
writing out July FRP_total rasters
writing out August AFC_num rasters
writing out August AFC_perc rasters
writing out August FRP_mean rasters
writing out August FRP_max rasters
writing out August FRP_total rasters
writing out September AFC_num rasters
writing out September AFC_perc rasters
writing out September FRP_mean rasters
writing out September FRP_max rasters
writing out September FRP_total rasters
writing out October AFC_num rasters
writing out October AFC_perc rasters
writing out October FRP_mean rasters
writing out October FRP_max rasters
writing out October FRP_total rasters
writin

writing out February FRP_max rasters
writing out February FRP_total rasters
writing out March AFC_num rasters
writing out March AFC_perc rasters
writing out March FRP_mean rasters
writing out March FRP_max rasters
writing out March FRP_total rasters
writing out April AFC_num rasters
writing out April AFC_perc rasters
writing out April FRP_mean rasters
writing out April FRP_max rasters
writing out April FRP_total rasters
writing out May AFC_num rasters
writing out May AFC_perc rasters
writing out May FRP_mean rasters
writing out May FRP_max rasters
writing out May FRP_total rasters
writing out June AFC_num rasters
writing out June AFC_perc rasters
writing out June FRP_mean rasters
writing out June FRP_max rasters
writing out June FRP_total rasters
writing out July AFC_num rasters
writing out July AFC_perc rasters
writing out July FRP_mean rasters
writing out July FRP_max rasters
writing out July FRP_total rasters
writing out August AFC_num rasters
writing out August AFC_perc rasters
wri

writing out December AFC_perc rasters
writing out December FRP_mean rasters
writing out December FRP_max rasters
writing out December FRP_total rasters
on year 2017
writing out January AFC_num rasters
writing out January AFC_perc rasters
writing out January FRP_mean rasters
writing out January FRP_max rasters
writing out January FRP_total rasters
writing out February AFC_num rasters
writing out February AFC_perc rasters
writing out February FRP_mean rasters
writing out February FRP_max rasters
writing out February FRP_total rasters
writing out March AFC_num rasters
writing out March AFC_perc rasters
writing out March FRP_mean rasters
writing out March FRP_max rasters
writing out March FRP_total rasters
writing out April AFC_num rasters
writing out April AFC_perc rasters
writing out April FRP_mean rasters
writing out April FRP_max rasters
writing out April FRP_total rasters
writing out May AFC_num rasters
writing out May AFC_perc rasters
writing out May FRP_mean rasters
writing out May 